In [1]:
import Graph_Amazon_Movies as gam
import Amazon_Movie_Parser as amp
import numpy as np
import pandas as pd
from networkx.algorithms import bipartite
import networkx as nx
import Amazon_Movie_Parser as prs
import datetime
from numpy import savetxt
import Computations_debug as comp
import random

#import Predict_Movie as pm
import Predict_Movie_v1 as pm
#import Predict_Movie_v2 as pm
#import Predict_Movie_v3 as pm
#import Predict_Movie_v4 as pm
pd.set_option('display.max_colwidth', 0)

In [2]:
# if you run this in colab *** it has a very limited RAM 12GB if I'm not mistaken, so works for 20K not 40K
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
predict_amazon = pm.Amazon()
#predict_amazon.debug_mode = 'On'
#predict_amazon.file_type = 'pickle'
# **** we need to filter out the movies the person has already reviewed previously ****
# because the same movie can appear in the test data-set since some new users might review it this time

In [4]:
file_name = 'data/movies.txt'; n_movies = 50000; n_movies_v = 20500; n_test_users = 100; walk_steps = 40
start_index_v = n_movies; beta = 0.1; top_neighbor = 50; top_N_movie_suggestions = 1

#file_name = 'data/movies.txt'; n_movies = 2000; n_movies_v = 2000; n_test_users = 40; walk_steps = 30
#start_index_v = n_movies; beta = 0.15; top_neighbor = 50

#lst_n_movies_test = [20000, 30000, 40000, 50000]
#lst_n_movies_test_v = [10000, 11000, 12000, 13000, 14000]
#lst_n_movies_test_v = [2000, 4000, 6000, 8000]
lst_n_movies_test_v = [20500]

start_time_ALL = datetime.datetime.now()
#for n_movies in lst_n_movies_test:
for n_movies_v in lst_n_movies_test_v:
    start_index_v = n_movies
    start_time_part1 = datetime.datetime.now()

    print('\nn_movies = {} .. n_movies_v = {} .. n_test_users = {}'.format(n_movies, n_movies_v, n_test_users))
    print('walk_steps = {} .. top_neighbor = {}'.format(walk_steps, top_neighbor))
    print('----------------------------------------------')
    print('Calculation starts for above values: {}'.format(start_time_part1))

    max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL = \
    predict_amazon.create_graphs(file_name, n_movies, n_movies_v, start_index_v)

    end_time_part1 = datetime.datetime.now()
    total_time_part1 = end_time_part1 - start_time_part1
    print('**********************************************')
    print('Calculation time for ALL-Part-1: {}'.format(total_time_part1))
    
    start_time_part2 = datetime.datetime.now()

    '''
    df_FINAL = \
    predict_amazon.run_validation(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor)
    '''
    '''
    df_FINAL = \
    predict_amazon.run_validation_TOP_N(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor, top_N_movie_suggestions)
    
    '''
    
    df_FINAL, df_FINAL_all = \
    predict_amazon.run_validation_TOP_N(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor, top_N_movie_suggestions)

    end_time_part2 = datetime.datetime.now()
    total_time_part2 = end_time_part2 - start_time_part2
    
    #start_time_part3 = datetime.datetime.now()
    # df_perf is saved in a csv, too, but it i done in the function "save_performance_values"
    df_perf = predict_amazon.save_performance_values(df_FINAL, total_time_part1, total_time_part2)
    
    df_perf_all = predict_amazon.save_performance_values_ALL(df_FINAL_all, total_time_part1, total_time_part2)
    #end_time_part3 = datetime.datetime.now()
    #total_time_part3 = end_time_part3 - start_time_part3

    # just to avoid failure if there is no RESULTS directory
    try:
        file_prefix = "RESULTS/PredictionDetails_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
        
        file_prefix_all = "RESULTS/PredictionDetails_ALL_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
    except:
        file_prefix = "PredictionDetails_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
        
        file_prefix_all = "PredictionDetails_ALL_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
                                                
    #fileName_Details = prs.FileNameUnique(prefix = "RESULTS/PredictionDetails_", suffix = '.csv')
    fileName_Details = prs.FileNameUnique(prefix = file_prefix, suffix = '.csv')
    fileName_Details_all = prs.FileNameUnique(prefix = file_prefix_all, suffix = '.csv')
    
    df_FINAL.to_csv(fileName_Details)
    df_FINAL_all.to_csv(fileName_Details_all)
    print('Prediction Details are saved in {}'.format(fileName_Details))
    print('Prediction_all Details are saved in {}'.format(fileName_Details_all))
    df_perf

end_time_ALL = datetime.datetime.now()
print('Calculation time for ALL-Predictions: {}'.format(end_time_ALL))

# it is taking SOOOOOO LONG time, had to stop it (even for 12K edges). Let's try in smaller networks later..
#d_model = nx.diameter(max_connected_gr_amazon_movies)
#d_test = nx.diameter(max_connected_gr_amazon_movies)

#print('Diameter of MODEL: {}\tTEST: {}'.format(d_model, d_test))

'''
>>>> df_FINAL (PredictionDetails.csv file)
> prediction: our prediction whether or not the given user will review the given movie on the list or not 
depending on the given threshold
> reality: 
> threshold: if ngbr_ratio > threshold then prediction=1 else 0
> ngbr_ratio: how many percent of your top-K similar users reviewed this movie
> tot_rev_m: how many times was this movie reviewed 
> n_old_edge: how many times did this user make a review in the old dataset
> n_new_edge: how many times did this user make a review in the new dataset
'''


n_movies = 50000 .. n_movies_v = 20500 .. n_test_users = 100
walk_steps = 40 .. top_neighbor = 50
----------------------------------------------
Calculation starts for above values: 2020-05-30 10:32:06.478843
*************** 1st Graph calculations - MODEL *************** 
Calculation time-Crete graph from the movies file: 0:01:02.129712
Is connected: True ... Is bipartite: True
# of nodes: 35940 ... # of edges: 47984
Calculation time-1st Part: 0:01:03.829501

*************** 2nd Graph calculations - TEST *************** 
Create_Bipartite_VALIDATION is running...
Calculation time-Crete graph from the movies file: 0:00:09.143592
Is connected: True ... Is bipartite: True
# of nodes: 15829 ... # of edges: 19293
n_movies for modeling (P matrix) = 1019 .. n_movies for testing = 357
n_users for modeling (P matrix) = 34921 .. n_users for testing = 15472
Avg degree (edge) per user in MODELING GRAPH: 1.3740729074196043  ... in TEST GRAPH: 1.2469622543950363
Calculation time-2nd Part: 0:00:10.20

'\n>>>> df_FINAL (PredictionDetails.csv file)\n> prediction: our prediction whether or not the given user will review the given movie on the list or not \ndepending on the given threshold\n> reality: \n> threshold: if ngbr_ratio > threshold then prediction=1 else 0\n> ngbr_ratio: how many percent of your top-K similar users reviewed this movie\n> tot_rev_m: how many times was this movie reviewed \n> n_old_edge: how many times did this user make a review in the old dataset\n> n_new_edge: how many times did this user make a review in the new dataset\n'

In [5]:
params_df = predict_amazon.show_parameters_in_use()
params_df.style.set_properties(**{'text-align': 'left'})

df_perf

,AUC,TPR,Precision,FPR,Thresholds,Calc_Time,lst_name,lst_value,other_metrics,metrics_val
0,0.925781,0.000000,0.000000,0,101,Graph Creation,debug_mode,Off,threshold,40
1,,0.722222,0.000000,0,100,0:01:14.039912,n_movies_model,50000,acc,0.92
2,,0.805556,1.000000,0,40,Random Walk & Prediction,n_movies_validation,20500,tpr,0.777778
3,,0.805556,1.000000,0.015625,30,0:58:59.146140,walk_steps,40,fpr,0
4,,0.833333,0.966667,0.046875,28,Threshold search,beta,0.1,tnr,1
5,,0.833333,0.909091,0.109375,26,0:00:00.719546,top_neighbor,50,fnr,0.222222
6,,0.861111,0.810811,0.140625,24,TOTAL TIME,n_test_users,100,precision,1
7,,0.861111,0.775000,0.1875,22,1:00:13.905598,threshold,40,f1,0.875
8,,0.888889,0.720930,0.3125,20,,start_index_v,50000,,
9,,0.888889,0.615385,0.359375,18,,file_name,data/movies.txt,,


In [6]:
df_perf_all

,AUC,TPR,Precision,FPR,Thresholds,Calc_Time,lst_name,lst_value,other_metrics,metrics_val
0,0.794235,0.000000,0.000000,0.000000,101.0,Graph Creation,debug_mode,Off,threshold,40
1,,0.171053,0.000000,0.000000,100.0,0:01:14.039912,n_movies_model,50000,acc,0.996527
2,,0.190789,1.000000,0.000000,40.0,Random Walk & Prediction,n_movies_validation,20500,tpr,0.184211
3,,0.190789,1.000000,0.000028,30.0,0:58:59.146140,walk_steps,40,fpr,0
4,,0.197368,0.966667,0.000113,28.0,Threshold search,beta,0.1,tnr,1
5,,0.197368,0.882353,0.000281,26.0,0:10:20.577112,top_neighbor,50,fnr,0.815789
6,,0.210526,0.750000,0.000647,24.0,TOTAL TIME,n_test_users,100,precision,1
7,,0.210526,0.581818,0.000900,22.0,1:10:33.763164,threshold,40,f1,0.311111
8,,0.217105,0.500000,0.001660,20.0,,start_index_v,50000,,
9,,0.243421,0.358696,0.002476,18.0,,file_name,data/movies.txt,,


In [7]:
# it is taking SOOOOOO LONG time
#d_model = nx.diameter(max_connected_gr_amazon_movies)
#d_test = nx.diameter(max_connected_gr_amazon_movies_VAL)
#print('Diameter of MODEL: {}\tTEST: {}'.format(d_model, d_test))

# centrality is fast
#centrality = nx.eigenvector_centrality_numpy(max_connected_gr_amazon_movies_VAL)
#print(centrality)

# if we use a small k, it doesn't take much time
#bc = nx.betweenness_centrality(max_connected_gr_amazon_movies_VAL, k=100)
#print(bc)


In [8]:
len(max_connected_gr_amazon_movies.edges('A1N0LCPR7O7OLL'))

4

In [9]:
import statistics as st
import networkx as nx
from networkx.algorithms import bipartite

bottom_nodes, top_nodes = bipartite.sets(max_connected_gr_amazon_movies_VAL)
# top_nodes=movies   ...  bottom_nodes=users
degree_centrality = bipartite.degree_centrality(max_connected_gr_amazon_movies_VAL, top_nodes)
#betweenness_centrality = bipartite.betweenness_centrality(max_connected_gr_amazon_movies_VAL, top_nodes)
#closeness_centrality = bipartite.closeness_centrality(max_connected_gr_amazon_movies_VAL, top_nodes)
#st.mean(list(degree_centrality.values()))
st.mean(list(betweenness_centrality.values()))
#---- degree_centrality (normalized) = deg(v)/max_possible_edges
# bottom-nodes: avg normalized-degree_centrality: 0.003492891468893659=VAL  ....  0.001348452313463792
# top-nodes: avg normalized-degree_centrality:    0.003492891468893659=VAL  ....  0.001348452313463792
#---- closeness_centrality
#---- https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.bipartite.centrality.closeness_centrality.html#networkx.algorithms.bipartite.centrality.closeness_centrality
# bottom-nodes: avg. closeness_centrality: ?=VAL                   ....  ?
# top-nodes: avg. closeness_centrality:    0.3937692751347393=VAL  ....  ?
#---- betweenness_centrality
# even if select bottom or top nodes, it always gives the result for bottom_nodes=USERS
# bottom-nodes: avg. betweenness_centrality: 1.2469622543950363=VAL  (just gave the avg. degree..)  ....  ?
# top-nodes: avg. betweenness_centrality:    1.2469622543950363=VAL  ....  ?
#----
# degree distribution
#degrees(B, nodes[, weight])
# careful here.. if you pass top_nodes as an argument, 
# it returns the values for bottom_nodes as the first result and the top_nodes as the second result.. and vice versa
#degree_distribution_Bottom, degree_distribution_TOP = bipartite.degrees(max_connected_gr_amazon_movies_VAL, top_nodes)
#print(len(degree_distribution_TOP))
#----
#bipartite.color(max_connected_gr_amazon_movies_VAL)
#----
# returns the same for both bottom and top... same value with avg normalized-degree_centrality
#dens_m = bipartite.density(max_connected_gr_amazon_movies_VAL, bottom_nodes)
#print(dens_m)
#---- clustering coeff for the whole graph
#avg_cluster = bipartite.average_clustering(max_connected_gr_amazon_movies_VAL, bottom_nodes)
#print(avg_cluster)
# 1.2469622543950363 >> (just gave the avg. degree..)
#----
#n_redundancy = bipartite.node_redundancy(max_connected_gr_amazon_movies_VAL, bottom_nodes)
#print(n_redundancy)
# NetworkXError: Cannot compute redundancy coefficient for a node that has fewer than two neighbors.
print('{} .. {}'.format(len(top_nodes), len(bottom_nodes)))
357*0.003492891468893659


NameError: name 'betweenness_centrality' is not defined

In [ ]:
#predict_amazon.df_summary_ratio_v.query("movie=='B000N2HD7I'").values[0].tolist()[1]
pd.options.display.max_rows = 1000
predict_amazon.df_summary_ratio_v

In [ ]:
# this takes too much time to run, as well
# The eccentricity of a node v is the maximum distance from v to all other nodes in G.
# Returns: ecc – A dictionary of eccentricity values keyed by node.
# eccentricity(G, v=None, sp=None)
#e_model = nx.eccentricity(max_connected_gr_amazon_movies)
#e_test = nx.eccentricity(max_connected_gr_amazon_movies_VAL)

#print('Eccentricity of MODEL: {}\tTEST: {}'.format(e_model, e_test))

In [1]:
# below table shows the prediction for threshold>10% so don't pay attention to it
# also tot_rev_m_NEW & OLD will be double checked, seems like a bug
pd.options.display.max_rows = 1000
df_FINAL

NameError: name 'pd' is not defined

In [ ]:
#pd.options.display.max_rows = 35500
df_FINAL_all.query("user=='A22GKUN35R09KN' and movie=='B000J6I0TS'")

In [ ]:
'''
total_reviews = 0
        # total number of reviews that those 100 test users did in dataset-2
        for u in df_FINAL['user'].unique():
            n_movie_reviewed_new = df_FINAL.query("user == '" + u + "'")['n_new_edge'][0]
            total_reviews += n_movie_reviewed_new



for u in df_FINAL['user'].unique():
    print(u)
    print(df_FINAL.query("user == '" + u + "'")['n_new_edge'].values[0])
'''

In [ ]:
len(df_FINAL['movie'].unique())

In [ ]:
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [ ]:
'''
for t in thresholds:
    acc, tpr, fpr, tnr, fnr, ppv, f1 = predict_amazon.final_summary_threshold_based(df_FINAL, t)
    lst_t = [t,  acc, tpr, fpr, tnr, fnr, ppv, f1]
    lst_presicion_based.append(lst_t)
'''

In [ ]:
'''
df_FINAL_copy = df_FINAL.copy()

threshold = 4
for i in range(len(df_FINAL_copy)):
    if df_FINAL_copy.loc[i, 'ngbr_ratio'] > threshold:
        df_FINAL_copy.loc[i, 'prediction'] = 1
    else:
        df_FINAL_copy.loc[i, 'prediction'] = 0
        
acc, tpr, fpr, tnr, ppv, f1 = predict_amazon.final_summary(df_FINAL_copy)

print(tpr)

#p_recall = tpr / (tpr + fnr)
#print(recall)
'''

In [ ]:
lst_true_val = df_FINAL.loc[:, 'reality'].values.tolist()
lst_ratio_val = df_FINAL.loc[:, 'ngbr_ratio'].values.tolist()

fpr, tpr, thresholds = metrics.roc_curve(lst_true_val, lst_ratio_val)  #, pos_label=2
lst_tpr = list(tpr); lst_fpr = list(fpr); lst_thresholds = list(thresholds) 

print(tpr)
print(thresholds)

In [ ]:
'''

y_true = df_FINAL.loc[:, 'reality'].values.tolist()
y_pred = df_FINAL.loc[:, 'ngbr_ratio'].values.tolist()

roc_auc = roc_auc_score(y_true, y_pred)
print(roc_auc)

#precision = precision_score(y_true, y_pred, average='macro') # 'micro'  'weighted' None  average=None, zero_division=1
#print(precision)

precision_recall = precision_recall_curve(y_true, y_pred)
#print(list(precision_recall))

avg_precision = average_precision_score(y_true, y_pred)
#print(avg_precision)

#lst_tpr = 

#for i in range(len(lst_tpr)):
 #   lst_precision[i] = lst_tpr[i]/(lst_tpr[i] + lst_fpr[i])
    
#print(lst_precision)
'''

In [ ]:
'''
df_FINAL_copy = df_FINAL.copy()

threshold = 4
for i in range(len(df_FINAL_copy)):
    if df_FINAL_copy.loc[i, 'ngbr_ratio'] >= threshold:
        df_FINAL_copy.loc[i, 'prediction'] = 1
    else:
        df_FINAL_copy.loc[i, 'prediction'] = 0

tp = len(df_FINAL_copy.query("prediction==1 and reality==1").values)
fp = len(df_FINAL_copy.query("prediction==1 and reality==0").values)
tn = len(df_FINAL_copy.query("prediction==0 and reality==0").values)
fn = len(df_FINAL_copy.query("prediction==0 and reality==1").values)

p_precise = tp / (tp + fp)
p_recall = tp / (tp + fn)

print(p_precise)
print(p_recall)
'''

In [ ]:
#u1=df_FINAL_copy.query("(prediction>0 or reality>0) and n_new_edge>0")
#u1=df_FINAL_copy.query("user=='AR15V2ULA2EYM' and (prediction>0 or reality>0) and n_new_edge>0 ").sort_values(by='ngbr_ratio', ascending=False)
#u1

In [ ]:
## NOTES
'''
Model1: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 80.23
Model2: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 86.89 .. 71.15 .. 
Model3: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 
Model4: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 

'''